# LLM Newspaper Comments Generation with LangChain and OpenAI

Giskard is an open-source framework for testing all ML models, from LLMs to tabular models. Don’t hesitate to give the project a [star on GitHub](https://github.com/Giskard-AI/giskard) ⭐️ if you find it useful!

In this notebook, you’ll learn how to create comprehensive test suites for your model in a few lines of code, thanks to Giskard’s open-source Python library.

In this example, we illustrate the procedure using **OpenAI Client** that is the default one; however, please note that our platform supports a variety of language models. For details on configuring different models, visit our [🤖 Setting up the LLM Client page](../../sdk/index.rst)

This notebook presents how to implement a LLM newspaper comments generation with Langchain and OpenAI embeddings.

Use-case:
  
* Newspaper comments generation

Outline:

* Detect vulnerabilities automatically with Giskard's scan
* Automatically generate & curate a comprehensive test suite to test your model beyond accuracy-related metrics

## Install dependencies

Make sure to install the `giskard[llm]` flavor of Giskard, which includes support for LLM models.

In [ ]:
%pip install "giskard[llm]" --upgrade

We also install the project-specific dependencies for this tutorial.

In [ ]:
%pip install "openai>=1" --upgrade

## Import libraries

In [1]:
import os

import openai
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAI

from giskard import Dataset, Model, scan

## Notebook settings

In [ ]:
# Set the OpenAI API Key environment variable.
OPENAI_API_KEY = "..."
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Display options.
pd.set_option("display.max_colwidth", None)

## Define constants

In [3]:
DATA_URL = "https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv"

TEXT_COLUMN_NAME = "text"

PROMPT_TEMPLATE = """
'{text}' \n\n 
As reader you have to critisize the authors of the article above starting now : I believe this article is really 
"""

RANDOM_STATE = 11

## Dataset preparation

### Load and preprocess data

In [ ]:
df = pd.read_csv(DATA_URL)
df_filtered = df[[TEXT_COLUMN_NAME]].sample(
    10, random_state=RANDOM_STATE, ignore_index=True
)

### Wrap dataset with Giskard

To prepare for the vulnerability scan, make sure to wrap your dataset using Giskard's Dataset class. More details [here](https://docs.giskard.ai/en/stable/open_source/scan/scan_tabular/index.html#step-1-wrap-your-dataset).

In [ ]:
giskard_dataset = Dataset(df_filtered, target=None)

## Model building

### Create an LLM Model with LangChain

In [ ]:
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=[TEXT_COLUMN_NAME],
)

llm = OpenAI(
    request_timeout=20,
    max_retries=100,
    temperature=0,
    model_name="gpt-3.5-turbo-instruct",
)

chain = LLMChain(prompt=prompt, llm=llm)

# Test the chain.
chain.invoke(df_filtered.loc[0, TEXT_COLUMN_NAME])

## Detect vulnerabilities in your model

### Wrap model with Giskard

To prepare for the vulnerability scan, make sure to wrap your model using Giskard's Model class. You can choose to either wrap the prediction function (preferred option) or the model object. More details [here](https://docs.giskard.ai/en/stable/open_source/scan/scan_tabular/index.html#step-2-wrap-your-model).

In [ ]:
giskard_model = Model(
    model=chain,
    model_type="text_generation",
    name="Comment generation",
    description="This model is a professional newspapers commentator.",
    feature_names=[TEXT_COLUMN_NAME],
)

Let’s check that the model is correctly wrapped by running it:

In [ ]:
# Validate the wrapped model and dataset.
print(giskard_model.predict(giskard_dataset).prediction)

### Scan your model for vulnerabilities with Giskard

We can now run Giskard's `scan` to generate an automatic report about the model vulnerabilities. This will thoroughly test different classes of model vulnerabilities, such as harmfulness, hallucination, prompt injection, etc.

The scan will use a mixture of tests from predefined set of examples, heuristics, and LLM based generations and evaluations.

Note: this can take up to 30 min, depending on the speed of the API.

Note that the scan results are not deterministic. In fact, LLMs may generally give different answers to the same or similar questions. Also, not all tests we perform are deterministic.

In [ ]:
results = scan(giskard_model, giskard_dataset)

In [9]:
display(results)

<iframe id="scan-5905542464" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collapse:col

## Generate comprehensive test suites automatically for your model

### Generate test suites from the scan

The objects produced by the scan can be used as fixtures to generate a test suite that integrates all detected vulnerabilities. Test suites allow you to evaluate and validate your model's performance, ensuring that it behaves as expected on a set of predefined test cases, and to identify any regressions or issues that might arise during development or updates.

In [10]:
test_suite = results.generate_test_suite("Test suite generated by scan")
test_suite.run()

2024-05-29 16:13:30,023 pid:89751 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
2024-05-29 16:13:30,026 pid:89751 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.021029
2024-05-29 16:13:30,032 pid:89751 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'text': 'object'} to {'text': 'object'}
2024-05-29 16:13:30,034 pid:89751 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:00:00.007765
Executed 'Basic Sycophancy' with arguments {'model': <giskard.models.langchain.LangchainModel object at 0x15fff6ec0>, 'dataset_1': <giskard.datasets.base.Dataset object at 0x32ffdaa40>, 'dataset_2': <giskard.datasets.base.Dataset object at 0x32ffd9180>}: 
               Test failed
               Metric: 1
               
               
2024-05-29 16:13:42,716 pid:89751 MainThread giskard.datasets.base INFO 

<TestSuiteResult (failed)>